# DD2417 Text Summarizer

## Imports

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    pipeline,
)
from datasets import load_dataset
import torch
import numpy as np
import json
from tqdm import tqdm

/home/jovyan/summarizer/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jovyan/summarizer/.env/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


# Setup

In [2]:
model_path = "./base/checkpoint-16500"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
summarizer = pipeline(
    "summarization",
    model=model,
    tokenizer=tokenizer,
)
dataset = load_dataset(
    "wikihow",
    "sep",
    data_dir="./data",
    split="validation",
    trust_remote_code=True,
)
np.random.seed(1)
samples = len(dataset)
n = 20
idx = np.random.choice(samples, size=n, replace=False)
dataset = dataset.select_columns(["text", "headline"])
dataset = dataset.select(idx)
dataset_dict = dataset.to_dict()

### Dataset

In [3]:
generated = []
for text in tqdm(dataset_dict["text"]):
    summary = summarizer(text, max_length=50, min_length=3, do_sample=False)
    summary_text = summary[0]['summary_text']
    generated.append(summary_text)
dataset_dict["generated"] = generated
with open("manual_eval.json", "w+") as f:
    json.dump(dataset_dict, f, indent=4)

100%|██████████| 50/50 [02:25<00:00,  2.92s/it]


### Evaluation